In [1]:
df <- read.csv("fifo.csv", header = TRUE, sep = ",")
#df$tally_q <- as.integer(df$tally_q)
# df$sale <- as.integer(df$sale)
# df$date <- as.Date(df$date, origin = "1899-12-30")

df$cf_q <- as.integer(rep(0, nrow(df)))
df$cf_b <- as.numeric(rep(0, nrow(df)))
df$sell_q <- as.integer(rep(0, nrow(df)))
df$sell_p <- as.numeric(rep(0, nrow(df)))
df$index <- seq(1:nrow(df))


head(df,3)
tail(df,3)

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,cf_q,cf_b,sell_q,sell_p,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<int>
1,12/31/2022,Buy,1,0,AMC,1050,4.07,4273.5,1050,4273.5,0,0,0,0,0,1
2,1/27/2023,Sell,1,1,AMC,-1000,5.51,-5512.4,50,-5512.4,-1000,0,0,0,0,2
3,1/20/2023,Buy,2,1,AMC,1000,5.63,5625.0,1050,5625.0,0,0,0,0,0,3


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,cf_q,cf_b,sell_q,sell_p,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<int>
39,5/17/2023,Buy,23,16,AMC,1000,5.01,5009.0,2100,5009.0,0,0,0,0,0,39
40,5/30/2023,Sell,23,17,AMC,-1000,4.58,-4582.8,1100,-4582.8,-1000,0,0,0,0,40
41,45079.00,Sell,23,18,AMC,-1100,4.54,-4994.0,0,-4994.0,-1100,0,0,0,0,41


In [2]:
# create seperate buy / sell dfs

buys <- df[df$quan > 0, ]  # Filter buys
sells <- df[df$quan < 0, ]  # Filter sells
df_pl <- data.frame()

In [3]:
# create vars
buy_qty <- buys[1, 'quan']
sell_qty <- abs(sells[1, 'quan'])

buy_basis <- buys[1, 'basis']
sell_basis <- abs(sells[1, 'basis'])


# sell_quan_remainder <- buy_qty - abs(sell_qty)

cat('sell_qty : ', sell_qty, "\n")
cat('buy_qty : ', buy_qty, "\n")
cat('buy_basis :', buy_basis, "\n")
cat('sell_basis :', sell_basis, "\n")
# cat('remainder :',sell_quan_remainder , "\n")

# zero out vars
sells[1, 'tally_q'] <- 0
sells[1, 'tally_b'] <- 0
sells[1, 'Sale'] <- 0

sell_qty :  1000 
buy_qty :  1050 
buy_basis : 4273.5 
sell_basis : 5512.4 


In [4]:
head(buys, 3)
head(sells, 3)

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,cf_q,cf_b,sell_q,sell_p,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<int>
1,12/31/2022,Buy,1,0,AMC,1050,4.07,4273.5,1050,4273.5,0,0,0,0,0,1
3,1/20/2023,Buy,2,1,AMC,1000,5.63,5625.0,1050,5625.0,0,0,0,0,0,3
5,1/23/2023,Buy,3,2,AMC,1000,5.79,5788.0,1050,5788.0,0,0,0,0,0,5


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,cf_q,cf_b,sell_q,sell_p,index,Sale
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>
2,1/27/2023,Sell,1,1,AMC,-1000,5.51,-5512.40,0,0.00,-1000,0,0,0,0,2,0
4,2/3/2023,Sell,2,2,AMC,-1000,6.68,-6682.55,50,-6682.55,-1000,0,0,0,0,4,NA
6,2/3/2023,Sell,3,3,AMC,-1000,6.26,-6255.00,50,-6255.00,-1000,0,0,0,0,6,NA



## <font color = yellow> Keeper for concept above

In [5]:
df_sales <- data.frame()

# add index column

df$index <- seq(1, nrow(df))

buys <- df[df$quan > 0, ]  # Filter buys
sells <- df[df$quan < 0, ]  

### <font color= yellow> # Review for user prompt 

In [6]:
# input_sell <- readline(prompt="Enter sale quantity: ")
# sell_quan <- as.numeric(input_sell)
# price <- readline(prompt="Enter sell price: ")
# sell_price <- as.numeric(price)

# str(sell_quan)
# str(sell_price)


In [7]:
for(i in 1:4) {

repeat {
      
        if(df[i, "type"] == "Sell"){             
            cat("Got a Sell!", "\n")
            cat("Row number ", i, " Selling ", abs(df$quan[i]), "quantity of shares. ", "\n")
            cat("Index number is: ", df[i, "index"], "\n")
            sell_quan <- df[i, 'quan']
            sell_price <- df[i, 'pps']
            # return(sell_quan)
            break
        }
        i <- i + 1
     }
}

Got a Sell! 
Row number  2  Selling  1000 quantity of shares.  
Index number is:  2 
Got a Sell! 
Row number  2  Selling  1000 quantity of shares.  
Index number is:  2 
Got a Sell! 
Row number  4  Selling  1000 quantity of shares.  
Index number is:  4 
Got a Sell! 
Row number  4  Selling  1000 quantity of shares.  
Index number is:  4 


In [8]:


nrow(sells)

     if(buys[1, 'quan'] > sell_quan) {       
         
     # create New Balance
         buys[1,'quan'] <- buys[1,'quan'] - sell_quan
         buys[1,'basis'] <- buys[1, 'quan'] * buys[1, "pps"]
         buys[1,'tally_q'] <- sell_quan
         buys[1,'sale'] <- buys[1,'sale'] + sell_quan * sell_price
         temp <- data.frame(buys[1, ])
         df_sales <- rbind(df_sales, temp)
         sells <- sells[-1, ]
    } else if (buys[1, 'quan']  < sell_quan) {       
         cf1 <- buys[1,'quan']
         cf2 <- buys[1, "basis"]
         buys <- buys[-1, ]
         buys[1,'quan'] <- buys[1,'quan'] + cf1
         buys[1,'basis'] <- buys[1, 'basis'] + cf2
         
         buys[1,'quan'] <- buys[1,'quan'] - sell_quan
         buys[1,'basis'] <- buys[1, 'quan'] * buys[1, "pps"]
         buys[1,'tally_q'] <- sell_quan
         buys[1,'sale'] <- buys[1,'sale'] + sell_quan * sell_price
         temp <- data.frame(buys[1, ])
         df_sales <- rbind(df_sales, temp)
         sells <- sells[-1, ]
         }
    


nrow(sells)
head(buys, 3)     
head(df_sales)


[1] 18

[1] 17

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,cf_q,cf_b,sell_q,sell_p,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>
1,12/31/2022,Buy,1,0,AMC,2050,4.07,8343.5,-1000,4273.5,-6680,0,0,0,0,1
3,1/20/2023,Buy,2,1,AMC,1000,5.63,5625.0,1050,5625.0,0,0,0,0,0,3
5,1/23/2023,Buy,3,2,AMC,1000,5.79,5788.0,1050,5788.0,0,0,0,0,0,5


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,cf_q,cf_b,sell_q,sell_p,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>
1,12/31/2022,Buy,1,0,AMC,2050,4.07,8343.5,-1000,4273.5,-6680,0,0,0,0,1


In [9]:
head(sells,3)

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,cf_q,cf_b,sell_q,sell_p,index
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<int>
4,2/3/2023,Sell,2,2,AMC,-1000,6.68,-6682.55,50,-6682.55,-1000,0,0,0,0,4
6,2/3/2023,Sell,3,3,AMC,-1000,6.26,-6255.00,50,-6255.00,-1000,0,0,0,0,6
8,2/3/2023,Sell,4,4,AMC,-1000,6.19,-6190.10,1050,-6190.10,-1000,0,0,0,0,8


In [10]:


# input_sell <- readline(prompt="Enter sale quantity: ")
# sell_quan <- as.numeric(input_sell)
# price <- readline(prompt="Enter sell price: ")
# sell_price <- as.numeric(price)

# nrow(sells)




#      if(buys[1, 'quan'] > sell_quan) {       
         
#      # create New Balance
#          buys[1,'quan'] <- buys[1,'quan'] - sell_quan
#          buys[1,'basis'] <- buys[1, 'quan'] * buys[1,"pps"]
#          buys[1,'tally_q'] <- sell_quan
#          buys[1,'sale'] <- buys[1,'sale'] + sell_quan * sell_price
#          temp <- data.frame(buys[1, ])
#          df_sales <- rbind(df_sales, temp)
#          sells <- sells[-1, ]
#     } else if (buys[1, 'quan']  < sell_quan) {       
#          cf1 <- buys[1,'quan']
#          cf2 <- buys[1,"basis"]
#          buys <- buys[-1, ]
#          buys[1,'quan'] <- buys[1,'quan'] + cf1
#          buys[1,'basis'] <- buys[1, 'basis'] + cf2
         
#          buys[1,'quan'] <- buys[1,'quan'] - sell_quan
#          buys[1,'basis'] <- buys[1, 'quan'] * buys[1,"pps"]
#          buys[1,'tally_q'] <- sell_quan
#          buys[1,'sale'] <- buys[1,'sale'] + sell_quan * sell_price
#          temp <- data.frame(buys[1, ])
#          df_sales <- rbind(df_sales, temp)
#          sells <- sells[-1, ]
#          }
    
    

# nrow(sells)
# head(buys, 3)     
# head(df_sales)


In [11]:
df <- read.csv("fifo.csv", header = TRUE, sep = ",")
df$index <- seq(1:nrow(df))
df$cf_q <- as.integer(rep(0, nrow(df)))
df$cf_b <- as.numeric(rep(0, nrow(df)))
df$sell_q <- as.integer(rep(0, nrow(df)))
df$sell_p <- as.numeric(rep(0, nrow(df)))
df$index <- seq(1:nrow(df))



df[1, 'type'] <- "Buy"

df_sales <- data.frame()
buys <- df[df$quan > 0, ]  # Filter buys
sells <- df[df$quan < 0, ]  

In [12]:
buy_ctr <- as.integer(1)
head(buys,3)
head(sells,3)

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,index,cf_q,cf_b,sell_q,sell_p
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>
1,12/31/2022,Buy,1,0,AMC,1050,4.07,4273.5,1050,4273.5,0,1,0,0,0,0
3,1/20/2023,Buy,2,1,AMC,1000,5.63,5625.0,1050,5625.0,0,3,0,0,0,0
5,1/23/2023,Buy,3,2,AMC,1000,5.79,5788.0,1050,5788.0,0,5,0,0,0,0


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,index,cf_q,cf_b,sell_q,sell_p
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>
2,1/27/2023,Sell,1,1,AMC,-1000,5.51,-5512.40,50,-5512.40,-1000,2,0,0,0,0
4,2/3/2023,Sell,2,2,AMC,-1000,6.68,-6682.55,50,-6682.55,-1000,4,0,0,0,0
6,2/3/2023,Sell,3,3,AMC,-1000,6.26,-6255.00,50,-6255.00,-1000,6,0,0,0,0


In [13]:
df[i,]

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,index,cf_q,cf_b,sell_q,sell_p
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>
4,2/3/2023,Sell,2,2,AMC,-1000,6.68,-6682.55,50,-6682.55,-1000,4,0,0,0,0


## <font color = lime> Change: using search parameters 

#### <font color = yellow> Open-ended (non-condition) search for a Sell

In [14]:
#  Open-ended, no hard code condtion.  Gets sell information for conditionals 

get_sell <- function (x) {

# change    for(i in x:nrow(sells)) {
        
    for(i in seq(from = x, to = nrow(df))) {
        
    cat("\n", "In got_sell fct - for loop.  The passed value to this fct is: ", x, "The i value is: ", i, "\n")
        
    ct <- 0
     # repeat {
        if(df[i, "type"] != "Sell") {
            cat("No Sell on line number: ", i, "\n")            
        }else if(df[i, "type"] == "Sell"){             
            cat("Got a Sell!", "\n")
            cat("The for-loop's i row testing number is: ", i, " Selling ", abs(df$quan[i]), "quantity of shares. ", "\n")
            cat("The Sell row index number is: ", df[i, "index"], "\n \n")
            sell_index <<- df[i, "index"]
            sell_quan <<- df[i, 'quan']
            #return()
            break
        }  # if
        i <- i + 1
   #  }   # repeat
  } # for
}  # fct
    

In [15]:
get_sell(31)




 In got_sell fct - for loop.  The passed value to this fct is:  31 The i value is:  31 
No Sell on line number:  31 

 In got_sell fct - for loop.  The passed value to this fct is:  31 The i value is:  32 
No Sell on line number:  32 

 In got_sell fct - for loop.  The passed value to this fct is:  31 The i value is:  33 
No Sell on line number:  33 

 In got_sell fct - for loop.  The passed value to this fct is:  31 The i value is:  34 
No Sell on line number:  34 

 In got_sell fct - for loop.  The passed value to this fct is:  31 The i value is:  35 
Got a Sell! 
The for-loop's i row testing number is:  35  Selling  2000 quantity of shares.  
The Sell row index number is:  35 
 


#### <font color = yellow> Hard code a condition for search (buy/sell next to each other)
    

## <font color = yellow> This works for first 5 sales

In [16]:
# hard coding the condition with the if stmt.  Req's a Sell_quan previous existence. 
# This is the engine that keeps things working until the sale involving 6000 

for(i in 1:12) {

    if(df[i, 'type'] == "Buy" && df[i+1, 'type'] == "Sell") {     
        cat("Works to derive sequential (buy/sell) rows. One (Buy) is: " , i, ", Two (Sell) is: ", i+1, "\n")    
        }
    
    get_sell(i)   # fct call, passing i

        #  This works great ! 
    
        if(sell_quan >= df[i, 'quan']) {
                        
            if(sell_quan == df[i, 'quan']) {            
                temp <- df[i,]
                df_sales <- rbind(df_sales, temp)
                sells <- sells[-i, ]
            } # if 2
        }  # if 1 
 }  # for

Works to derive sequential (buy/sell) rows. One (Buy) is:  1 , Two (Sell) is:  2 

 In got_sell fct - for loop.  The passed value to this fct is:  1 The i value is:  1 
No Sell on line number:  1 

 In got_sell fct - for loop.  The passed value to this fct is:  1 The i value is:  2 
Got a Sell! 
The for-loop's i row testing number is:  2  Selling  1000 quantity of shares.  
The Sell row index number is:  2 
 

 In got_sell fct - for loop.  The passed value to this fct is:  2 The i value is:  2 
Got a Sell! 
The for-loop's i row testing number is:  2  Selling  1000 quantity of shares.  
The Sell row index number is:  2 
 
Works to derive sequential (buy/sell) rows. One (Buy) is:  3 , Two (Sell) is:  4 

 In got_sell fct - for loop.  The passed value to this fct is:  3 The i value is:  3 
No Sell on line number:  3 

 In got_sell fct - for loop.  The passed value to this fct is:  3 The i value is:  4 
Got a Sell! 
The for-loop's i row testing number is:  4  Selling  1000 quantity of shar

In [17]:
length(buys)
length(sells)

[1] 16

[1] 16

In [18]:


# input_sell <- readline(prompt="Enter sale quantity: ")
# sell_quan <- as.numeric(input_sell)
# price <- readline(prompt="Enter sell price: ")
# sell_price <- as.numeric(price)

sell_index <- as.integer()
sell_quan <- as.integer()



cat("sell_index is: ", sell_index, "\n")
cat("sell_quan is: ", sell_quan, "\n")

ct <- sell_index


for(i in 1:nrow(df)) {
    
    get_sell(i)
    
    buy_index <- i
    
    cat("top of for loop, i ia: ", i, "\n") 
    
    cat("buys and sells indexes' are : ", buy_index, " and ", sell_index, "\n")

# change buy_index == 0 to i
   
    if(sell_index - buy_index == i && abs(df[i+1, 'quan']) == df[i, 'quan']) {
       cat("This will be a simple wash sale with no c/f", "\n")
       # one possibility of eliminating these rows is to zero the quans and set a flag
        df[i, 'sell_q' ] <- sell_quan
        df[i, 'sell_p'] <- df[i+1, 'pps'] * sell_quan
        df[i +1, 'type'] <- "Sold"
        df[i +1, 'index'] <- i
        sell_quan <- 0
        sell_index <- 0
        cat("Clean sale. No C/F from sales condition type 1 ", "\n")
        return()       
        
    }
    
    else if(sell_index - buy_index == 1 && df[i, 'quan'] < 0) {
        cat("This will be a simple wash sale with no c/f", "\n")
        }
         
    else if(sell_index - buy_index ==1 && df[i, 'quan']  < df[i+1, 'quan'] ) {
        
        cat("This buy row gets entirely sold with additional sold from the next buy line(s) ", "\n")
         
        # cat("top of if # 1 ", "\n") 
        ct <- ct + buy_ctr   
         
         # one possibility of eliminating these rows is to zero the quans and set a flag
        df[i, 'sell_q' ] <- df[i, 'quan']
        df[i, 'sell_p'] <- df[i+1, 'pps'] * sell_quan
        df[i +1, 'type'] <- "Sold"
        df[i, 'type'] <- "Sold"
        df[i +1, 'index'] <- i
        df[i, 'index'] <- i
        sell_quan <- 0
        sell_index <- 0
        cat("clean, no C/F from sales condition type 2 ", "\n")
        return()       
        
                
    
         
        # when the buys row i has quanity < sell quan, need carry values forward to next row,
         # And mo
       } else if(sell_index - buy_index ==1 && df[i, 'quan']  > df[i+1, 'quan'] ) {
        
        cat("top of else if # 3 ", "\n") 
        
        
        # create allocation and c/f amounts
                
        df[i, 'cf_q'] <-     df[i, 'quan'] - abs(df[i+1, 'quan'])                                            
        df[i + 1, 'quan'] <- df[i, 'quan'] - abs(df[i+1, 'quan'])  # adj sell quan after removing partial sell
                                             
        df[i, 'cf_b'] <-    df[i, 'cf_q']  * df[i, 'pps']        
        df[i, 'sell_q' ] <- df[i, 'quan'] - df[i, 'cf_q']
        
        df[i, 'sell_p'] <- df[i+1, 'pps'] * (df[i, 'quan'] - abs(df[i+1, 'quan']) )
        df[i , 'type'] <- "Sold"
        df[i+1, 'type'] <- "Part Sold"
        
        df[i +1, 'index'] <- i
        df[i, 'index'] <- i
        sell_quan <- 0
        sell_index <- 0
        cat("Partial sale. Has C/F from sales condition type 3 ", "\n")
        # return()
        
        return       
        
         ct <- ct + i
        
         buys[i,'sale'] <- buys[i,'sale'] + sell_quan * sell_price
         temp <- data.frame(buys[i, ])
         df_sales <- rbind(df_sales, temp)
         sells <- sells[-i, ]
         }
    
   # print(buys[i,])

    }
    
    


head(df, 10)
head(df_sales)

sell_index is:   
sell_quan is:   

 In got_sell fct - for loop.  The passed value to this fct is:  1 The i value is:  1 
No Sell on line number:  1 

 In got_sell fct - for loop.  The passed value to this fct is:  1 The i value is:  2 
Got a Sell! 
The for-loop's i row testing number is:  2  Selling  1000 quantity of shares.  
The Sell row index number is:  2 
 
top of for loop, i ia:  1 
buys and sells indexes' are :  1  and  2 
top of else if # 3  
Partial sale. Has C/F from sales condition type 3  

 In got_sell fct - for loop.  The passed value to this fct is:  2 The i value is:  2 
No Sell on line number:  2 

 In got_sell fct - for loop.  The passed value to this fct is:  2 The i value is:  3 
No Sell on line number:  3 

 In got_sell fct - for loop.  The passed value to this fct is:  2 The i value is:  4 
Got a Sell! 
The for-loop's i row testing number is:  4  Selling  1000 quantity of shares.  
The Sell row index number is:  4 
 
top of for loop, i ia:  2 
buys and sells inde

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,index,cf_q,cf_b,sell_q,sell_p
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,12/31/2022,Sold,1,0,AMC,1050,4.07,4273.50,1050,4273.50,0,1,50,203.5,1000,5510
2,1/27/2023,Part Sold,1,1,AMC,50,5.51,-5512.40,50,-5512.40,-1000,1,0,0.0,0,0
3,1/20/2023,Sold,2,1,AMC,1000,5.63,5625.00,1050,5625.00,0,3,0,0.0,1000,6680
4,2/3/2023,Part Sold,2,2,AMC,0,6.68,-6682.55,50,-6682.55,-1000,3,0,0.0,0,0
5,1/23/2023,Sold,3,2,AMC,1000,5.79,5788.00,1050,5788.00,0,5,0,0.0,1000,6260
6,2/3/2023,Part Sold,3,3,AMC,0,6.26,-6255.00,50,-6255.00,-1000,5,0,0.0,0,0
7,1/27/2023,Sold,4,3,AMC,2000,5.24,10478.80,2050,10478.80,0,7,1000,5240.0,1000,6190
8,2/3/2023,Part Sold,4,4,AMC,1000,6.19,-6190.10,1050,-6190.10,-1000,7,0,0.0,0,0
9,2/10/2023,Sold,5,4,AMC,2000,4.75,9490.00,3050,9490.00,0,9,-1000,-4750.0,3000,5860


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,index,cf_q,cf_b,sell_q,sell_p
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>
2,1/27/2023,Sell,1,1,AMC,-1000,5.51,-5512.40,50,-5512.40,-1000,2,0,0,0,0
4,2/3/2023,Sell,2,2,AMC,-1000,6.68,-6682.55,50,-6682.55,-1000,4,0,0,0,0
6,2/3/2023,Sell,3,3,AMC,-1000,6.26,-6255.00,50,-6255.00,-1000,6,0,0,0,0
8,2/3/2023,Sell,4,4,AMC,-1000,6.19,-6190.10,1050,-6190.10,-1000,8,0,0,0,0
10,2/21/2023,Sell,5,5,AMC,-3000,5.86,-17565.00,50,-17565.00,-3000,10,0,0,0,0
1,12/31/2022,Buy,1,0,AMC,1050,4.07,4273.50,1050,4273.50,0,1,0,0,0,0


In [19]:
nrow(df)



[1] 41

In [20]:
df_sales


,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,index,cf_q,cf_b,sell_q,sell_p
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>
2,1/27/2023,Sell,1,1,AMC,-1000,5.51,-5512.40,50,-5512.40,-1000,2,0,0,0,0
4,2/3/2023,Sell,2,2,AMC,-1000,6.68,-6682.55,50,-6682.55,-1000,4,0,0,0,0
6,2/3/2023,Sell,3,3,AMC,-1000,6.26,-6255.00,50,-6255.00,-1000,6,0,0,0,0
8,2/3/2023,Sell,4,4,AMC,-1000,6.19,-6190.10,1050,-6190.10,-1000,8,0,0,0,0
10,2/21/2023,Sell,5,5,AMC,-3000,5.86,-17565.00,50,-17565.00,-3000,10,0,0,0,0
1,12/31/2022,Buy,1,0,AMC,1050,4.07,4273.50,1050,4273.50,0,1,0,0,0,0
5,1/23/2023,Buy,3,2,AMC,1000,5.79,5788.00,1050,5788.00,0,5,0,0,0,0
9,2/10/2023,Buy,5,4,AMC,2000,4.75,9490.00,3050,9490.00,0,9,0,0,0,0
12,2/13/2023,Buy,7,5,AMC,3000,4.73,14188.80,4050,14188.80,0,12,0,0,0,0


In [21]:
sells

,date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,index,cf_q,cf_b,sell_q,sell_p
,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>
6,2/3/2023,Sell,3,3,AMC,-1000,6.26,-6255.0,50,-6255.0,-1000,6,0,0,0,0
8,2/3/2023,Sell,4,4,AMC,-1000,6.19,-6190.1,1050,-6190.1,-1000,8,0,0,0,0
18,2/27/2023,Sell,11,7,AMC,-3000,7.82,-23448.0,2850,-23448.0,-3000,18,0,0,0,0
20,2/27/2023,Sell,11,9,AMC,-1000,7.82,-7816.0,0,-7816.0,-1000,20,0,0,0,0
27,3/8/2023,Sell,15,12,AMC,-3000,5.95,-17835.0,2500,-17835.0,-3000,27,0,0,0,0
28,3/8/2023,Sell,15,13,AMC,-2000,5.76,-11510.0,500,-11510.0,-2000,28,0,0,0,0
36,4/20/2023,Sell,20,16,AMC,-5000,4.95,-24725.0,0,-24725.0,-5000,36,0,0,0,0
40,5/30/2023,Sell,23,17,AMC,-1000,4.58,-4582.8,1100,-4582.8,-1000,40,0,0,0,0


In [23]:
df

date,type,buy_ct,sell_ct,co,quan,pps,basis,tally_q,tally_b,sale,index,cf_q,cf_b,sell_q,sell_p
<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
12/31/2022,Sold,1,0,AMC,1050,4.07,4273.50,1050,4273.50,0,1,50,203.5,1000,5510.0
1/27/2023,Part Sold,1,1,AMC,50,5.51,-5512.40,50,-5512.40,-1000,1,0,0.0,0,0.0
1/20/2023,Sold,2,1,AMC,1000,5.63,5625.00,1050,5625.00,0,3,0,0.0,1000,6680.0
2/3/2023,Part Sold,2,2,AMC,0,6.68,-6682.55,50,-6682.55,-1000,3,0,0.0,0,0.0
1/23/2023,Sold,3,2,AMC,1000,5.79,5788.00,1050,5788.00,0,5,0,0.0,1000,6260.0
2/3/2023,Part Sold,3,3,AMC,0,6.26,-6255.00,50,-6255.00,-1000,5,0,0.0,0,0.0
1/27/2023,Sold,4,3,AMC,2000,5.24,10478.80,2050,10478.80,0,7,1000,5240.0,1000,6190.0
2/3/2023,Part Sold,4,4,AMC,1000,6.19,-6190.10,1050,-6190.10,-1000,7,0,0.0,0,0.0
2/10/2023,Sold,5,4,AMC,2000,4.75,9490.00,3050,9490.00,0,9,-1000,-4750.0,3000,5860.0


In [22]:
stopper

ERROR: Error in eval(expr, envir, enclos): object 'stopper' not found


## <font color = yellow> Working code blocks below:


In [ ]:
for(i in 1:4) {
    df[which(df[i, "sell_ct"] == i & df[i, "type"] == "Sell"), ]
    
    cat("Row number ", i, "type is :", df[i, "type"], " The Sell qty: ", df$quan[i], " number of shares. ", "\n")
    
    }
    

In [ ]:
# df[which(df$B == value_to_find), ]

# df[which(df$A == value_to_find_A & df$B == value_to_find_B), ]


for(i in 1:4) {
    df[which(df$sell_ct == i & df$type == "Sell"), ]
    
    if(df[i,"type"] == "Sell") {
    cat("Row number ", i, " Selling ", df$quan[i], " number of shares. ", "\n")
    }
}


In [ ]:
ct <- 0 

for(i in 1:4) {

    repeat {
      
        if(df[i, "type"] == "Sell"){             
            cat("Got a Sell!", "\n")
            cat("Row number ", i, " Selling ", abs(df$quan[i]), "quantity of shares. ", "\n")
            cat("Index number is: ", df[i, "index"], "\n")
            break
        }
        i <- i + 1
     }
}


  


In [ ]:
x <- 1
df_pl <- data.frame()
pl <- function(x) {
    
    sells[x, 'tally_b'] <<- buys[x, 'basis']
    
    sells[x, 'Sale'] <<- abs(sells[x, 'basis']) - sells[x, 'tally_b']
    temp <- data.frame(sells[x,])
    df_pl <<- rbind(df_pl, temp)
    
    }
 pl(x)
 head(df_pl, 3)
# head(sells, 3)


#### <font color = yellow> Passing a value to fct, but the for loop determines increment or decrement the df
    
        decrement loop by 1
        for(i in seq(from = x, to = 1)) {
          # Your code here
         }
    
        increment the loop by 1 
        for(i in seq(from = x, to = nrow(df))) {
            # Your code here
        }
    
        

In [ ]:
# Gets sell information for conditionals 

get_sell <- function (x) {

    for(i in x:nrow(sells)) {
        
    cat("\n", "In got_sell fct - for loop.  The passed value to this fct is: ", x, "The i value is: ", i, "\n")
        
    ct <- 0
     # repeat {
        if(df[i, "type"] != "Sell") {
            cat("No Sell on line number: ", i, "\n")            
        }else if(df[i, "type"] == "Sell"){             
            cat("Got a Sell!", "\n")
            cat("The for-loop's i row testing number is: ", i, " Selling ", abs(df$quan[i]), "quantity of shares. ", "\n")
            cat("The Sell row index number is: ", df[i, "index"], "\n \n")
            sell_index <<- df[i, "index"]
            sell_quan <<- df[i, 'quan']
            #return()
            break
        }  # if
        i <- i + 1
   #  }   # repeat
  } # for
}  # fct
get_sell(31)   # this decrements because the nrows(Sells) is less than 31
